In [35]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd

import seaborn as sns

import matplotlib.pyplot as plt
from matplotlib import rc
rc('font', family='NanumGothic')
plt.rcParams['axes.unicode_minus'] = False

In [36]:
file = 'dataset_final_240201.csv'

dataset = pd.read_csv(file)
print(dataset.shape)
dataset.head().transpose()

(1750, 557)


,0,1,2,3,4
Unnamed: 0,0,1,2,3,4
종목코드,KR6503313555,KR6503313555,KR6503313555,KR6503322549,KR6503322549
차수,2.0,3.0,4.0,2.0,3.0
평가구분,중간평가,중간평가,중간평가,중간평가,중간평가
평가종료일,2016-05-10,2016-11-08,2017-05-09,2016-04-01,2016-10-04
...,...,...,...,...,...
홍콩파생상품(%)_shift,2.777778,1.934236,4.841897,-3.10559,7.484407
일일증권평균거래량_Main Board_Other equity stocks(%)_shift,2.323232,0.196464,-0.878906,-0.60241,0.493583
홍콩거래소시가총액_Main BoardH Shares(%)_shift,1.431493,0.785855,1.573255,-1.012146,2.620968
국채 수익률,1.91,1.77,2.28,1.91,1.7


In [37]:
cols_drop_info = ['종목코드', '기준가1', '녹인가1', '환매일 종가', '평가기준가']
cols_drop_duplicate = ['평가구분', '상환조건달성'] + ['녹인발생차수'] # unique 값이 1개
cols_drop_future = ['상환구분', '상환실현차수']

cols_drop_dt = ['발행일', '상환일', '평가시작일', '평가종료일', '환매결정일', '녹인발생일']
# 데이터 타입 변경
for col in cols_drop_dt:
    dataset[col] = pd.to_datetime(dataset[col])


# 피처로 쓰기 애매함
cols_cnt = [
    '녹인일수', '녹인일수_전', '영업일수', '상환일수'
]

cols_feature = ['차수', '기초자산개수', '녹인발생차수_차이']
cols_cat = ['환매일종가위치'] # 범주형 변수
cols_dummy = ['환매일종가위치_code']
dataset['환매일종가위치_code'] = dataset['환매일종가위치'].astype('category').cat.codes.astype(float)
cols_pct100 = [
    '상환조건(%)', '하한 수준(%)', '상환조건감소량(%)_prev', '상환조건감소량(%)_next',
    '환매일 수준(%)', '녹인대비상환수준(%)', '환매대비상환수준(%)', '환매대비상환수준(%)_next'
]
cols_pct = [
    '녹인비율', '녹인비율_전', 'H총증감률', 'H평균증감률', 'H일평균증감률', 'H이전대비증감률', '상환비율'
]
# 비율 단위 변경
for col in cols_pct:
    col_new = col+"(%)"
    dataset[col_new] = dataset[col]*100
    cols_pct100.append(col_new)

#
cols_drop = cols_drop_info + cols_drop_duplicate + cols_drop_future + cols_drop_dt + cols_cat


In [30]:
dataset_values = dataset.drop(columns=cols_drop+cols_pct)
dataset_values.shape

(1750, 533)

In [31]:
# dataset_values.corr()['label'].sort_values()*100

In [32]:
col_X = cols_feature + cols_pct100
col_y = 'label'

df = dataset[col_X + [col_y]]

X = dataset[col_X]
y = dataset[col_y]

df.shape, X.shape, y.shape

((1750, 19), (1750, 18), (1750,))

In [33]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
# Assuming cols_feature_long is your DataFrame
# Select only the numerical columns for scaling
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify= y)

columns_to_scale = ['Oman(%)',
 '국채 수익률_std',
 'cpi_ind(%)_shift',
 'Brent_std',
 'WTI(%)',
 '홍콩외환보유액(%)_diff',
 '예금금리_hk_std',
 '홍콩직접투자(%)_shift',
 '홍콩수입액_shift_diff',
 '상환비율(%)',
 'Dubai(%)_diff',
 '국채 수익률증감_diff',
 '상환조건감소량(%)_prev',
 '홍콩CPI_shift_diff()',
 'cpi_kor(%)_shift',
 '녹인비율_전(%)',
 '홍콩 M2 (2).1(%)_shift_diff',
 'Brent_diff',
 '홍콩CPI_shift_diff()_diff',
 '녹인대비상환수준(%)',
 '변동_diff',
 'cpi_jap(%)_shift',
 'msci_China(USD)(%)_diff',
 'Brent(%)_diff',
 '홍콩수입액_diff',
 '국채 수익률증감',
 '홍콩외환보유액_diff',
 'CLI_FRA_dummy_diff',
 '홍콩수출액_shift_diff_diff']

In [34]:
import pandas as pd
from sklearn.model_selection import train_test_split

X = X[['cpi_jap(%)_shift',
 '홍콩CPI_shift_diff()_diff',
 '홍콩수입액_shift_diff',
 '홍콩외환보유액_diff',
 'Brent_diff',
 '홍콩직접투자(%)_shift',
 '홍콩수출액_shift_diff_diff',
 'cpi_kor(%)_shift',
 '홍콩CPI_shift_diff()',
 'Brent_std',
 'cpi_ind(%)_shift',
 '홍콩 M2 (2).1(%)_shift_diff',
 '국채 수익률_std',
 'CLI_FRA_dummy_diff',
 '홍콩수입액_diff',
 '국채 수익률증감_diff',
 'Dubai(%)_diff',
 '홍콩외환보유액(%)_diff',
 'Brent(%)_diff',
 'msci_China(USD)(%)_diff',
 '국채 수익률증감',
 'Oman(%)',
 'WTI(%)',
 '녹인비율_전(%)',
 '녹인대비상환수준(%)',
 '예금금리_hk_std',
 '상환비율(%)',
 '상환조건감소량(%)_prev',
 '변동_diff',
 '기초자산개수']]

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify= y)

# Create an instance of StandardScaler
scaler = StandardScaler()

# Fit the scaler to the selected columns and transform them
scaler = scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_val = scaler.transform(X_val)

KeyError: "['cpi_jap(%)_shift', '홍콩CPI_shift_diff()_diff', '홍콩수입액_shift_diff', '홍콩외환보유액_diff', 'Brent_diff', '홍콩직접투자(%)_shift', '홍콩수출액_shift_diff_diff', 'cpi_kor(%)_shift', '홍콩CPI_shift_diff()', 'Brent_std', 'cpi_ind(%)_shift', '홍콩 M2 (2).1(%)_shift_diff', '국채 수익률_std', 'CLI_FRA_dummy_diff', '홍콩수입액_diff', '국채 수익률증감_diff', 'Dubai(%)_diff', '홍콩외환보유액(%)_diff', 'Brent(%)_diff', 'msci_China(USD)(%)_diff', '국채 수익률증감', 'Oman(%)', 'WTI(%)', '예금금리_hk_std', '변동_diff'] not in index"

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X_train, y_train)

# 예측
y_pred = model.predict(X_val)

# 모델 성능평가
from sklearn.metrics import confusion_matrix

cf_matrix = confusion_matrix(y_val, y_pred)
print(cf_matrix)

from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score

print("Accuracy : %.3f" % accuracy_score(y_val, y_pred))
print("Precision : %.3f" % precision_score(y_val, y_pred))
print("Recall : %.3f" % recall_score(y_val, y_pred))
print("F1 : %.3f" % f1_score(y_val, y_pred))

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=0)

# fit
dt.fit(X_train, y_train)

# 예측
y_pred = dt.predict(X_val)


# 모델 성능평가
from sklearn.metrics import confusion_matrix
cf_matrix = confusion_matrix(y_val, y_pred)
print(cf_matrix)


# score print
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
print("Accuracy : %.3f" % accuracy_score(y_val, y_pred))
print("Precision : %.3f" % precision_score(y_val, y_pred))
print("Recall : %.3f" % recall_score(y_val, y_pred))
print("F1 : %.3f" % f1_score(y_val, y_pred))

In [ ]:
from sklearn.svm import SVC
svc = SVC(random_state=0, probability=True)

# fit
svc.fit(X_train, y_train)

# 예측
y_pred = svc.predict(X_val)


# 모델 성능평가
from sklearn.metrics import confusion_matrix
cf_matrix = confusion_matrix(y_val, y_pred)
print(cf_matrix)


# score print
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
print("Accuracy : %.3f" % accuracy_score(y_val, y_pred))
print("Precision : %.3f" % precision_score(y_val, y_pred))
print("Recall : %.3f" % recall_score(y_val, y_pred))
print("F1 : %.3f" % f1_score(y_val, y_pred))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier()

# fit
rf_model.fit(X_train, y_train)

# 예측
y_pred = rf_model.predict(X_val)


# 모델 성능평가
from sklearn.metrics import confusion_matrix
cf_matrix = confusion_matrix(y_val, y_pred)
print(cf_matrix)


# score print
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
print("Accuracy : %.3f" % accuracy_score(y_val, y_pred))
print("Precision : %.3f" % precision_score(y_val, y_pred))
print("Recall : %.3f" % recall_score(y_val, y_pred))
print("F1 : %.3f" % f1_score(y_val, y_pred))

In [ ]:
from xgboost import XGBClassifier
xgb_model = XGBClassifier()

# fit
xgb_model.fit(X_train, y_train)

# 예측
y_pred = xgb_model.predict(X_val)


# 모델 성능평가
from sklearn.metrics import confusion_matrix
cf_matrix = confusion_matrix(y_val, y_pred)
print(cf_matrix)


# score print
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
print("Accuracy : %.3f" % accuracy_score(y_val, y_pred))
print("Precision : %.3f" % precision_score(y_val, y_pred))
print("Recall : %.3f" % recall_score(y_val, y_pred))
print("F1 : %.3f" % f1_score(y_val, y_pred))

In [ ]:
from lightgbm import LGBMClassifier
lgbm_model = LGBMClassifier()

# fit
lgbm_model.fit(X_train, y_train)

# 예측
y_pred = lgbm_model.predict(X_val)


# 모델 성능평가
from sklearn.metrics import confusion_matrix
cf_matrix = confusion_matrix(y_val, y_pred)
print(cf_matrix)


# score print
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
print("Accuracy : %.3f" % accuracy_score(y_val, y_pred))
print("Precision : %.3f" % precision_score(y_val, y_pred))
print("Recall : %.3f" % recall_score(y_val, y_pred))
print("F1 : %.3f" % f1_score(y_val, y_pred))

In [ ]:
def model_basic(X_train, y_train, X_test, y_test):
    models = [
        LogisticRegression(),
        DecisionTreeClassifier(),
        SVC(),
        RandomForestClassifier(),
        XGBClassifier(),
        LGBMClassifier()
    ]

    rdict={'model':[],'accuracy':[],'precision':[],'recall':[],'f1_score':[]}

    
    for clf in models:
        clf = clf.fit(X_train, y_train)
        pred = clf.predict(X_test)
        results = (round(accuracy_score(y_test, pred),2),
                    round(precision_score(y_test, pred),2),
                    round(recall_score(y_test, pred),2),
                    round(f1_score(y_test, pred),2))
        rdict['model'].append(clf); 
        rdict['accuracy'].append(results[0])
        rdict['precision'].append(results[1])
        rdict['recall'].append(results[2])
        rdict['f1_score'].append(results[3])

        # print(results)

    rdf = pd.DataFrame(data=rdict)
    return rdf 

In [ ]:
model_basic(X_train, y_train, X_val, y_val)